In [38]:
import pandas as pd
import geopandas as gpd
import numpy as np
from sklearn.preprocessing import Normalizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import math

In [39]:
dimensoes = [
    # 'situacao_setor',#
    # 'tipo_setor',#
    'numero_de_domicilios',#
    'numero_moradores',#
    'porcentagem_domicilios_tipo_comodo', #
    'porcentagem_domicilios_sem_coleta_de_lixo', #
    'porcentagem_domicilios_sem_ligacao_rede_de_agua', #
    'porcentagem_domicilios_sem_banheiros', #
    'porcentagem_domicilios_sem_esgoto_ou_fossa', #
    'porcentagem_domicilios_cedidos_outra_forma', #
    'numero_de_banheiros_por_habitante', #
    'porcentagem_responsaveis_nao_alfabetizado', #
    'porcentagem_responsaveis_nao_alfabetizado_com_menos_de_30_anos', #
    'porcentagem_de_responsaveis_com_menos_30_anos', #
    'porcentagem_responsaveis_renda_ate_3_salarios', #
    'porcentagem_responsaveis_menos_8_anos_de_estudo', #
    'anos_medios_estudo_responsavel', #
    'renda_media_responsavel', #
    'numero_de_domicilios_improvisados', #
    'numero_medio_pessoas', #
]

In [40]:
df = pd.read_csv('resultados/modelo-2000.csv.zip')
df.fillna(0, inplace=True)
# rmsp = (df.rm == 20)
subnormal = (df.tipo_setor == 1)
df.loc[:, 'subnormal'] = 0
df.loc[subnormal, 'subnormal'] = 1
df.loc[:, 'processado'] = False

In [41]:
df.tipo_setor.value_counts()

tipo_setor
0    206570
1      7871
7       578
5       387
6       219
2        96
3        87
4         3
Name: count, dtype: int64

In [42]:
r_subnormal = df.loc[(subnormal), ['rm', 'subnormal']].groupby('rm').count().reset_index()
cidades_fora_rms = r_subnormal[(r_subnormal.subnormal < 20) | (r_subnormal.rm == 0.)].rm.to_list()
rms = r_subnormal[(r_subnormal.subnormal >= 20) & (r_subnormal.rm != 0.)].rm.to_list()

In [43]:
rms

[1, 2, 3, 5, 6, 7, 8, 10, 13, 14, 15, 16, 17, 26, 27, 28]

In [44]:
df.loc[:, ['rm', 'nome_rm']].groupby('rm').first()

,nome_rm
rm,
0,Não pertence
1,Belém
2,Grande São Luís
3,Fortaleza
4,Natal
5,Recife
6,Maceió
7,Salvador
8,Belo Horizonte


In [103]:
def processa_modelos(filter, rm=None, uf=None):

    registro = {}

    # filter = (df.rm == rm)
    df_rm = df.loc[(filter)]
    df.loc[(filter), 'processado'] = True
    print(df.loc[(filter)].shape)

    # print(df_rm.nome_rm.unique()[0])
    if rm:
        registro['nome_rm_ou_UF'] = df_rm.nome_rm.unique()[0]
    
    if uf:
        registro['nome_rm_ou_UF'] = uf
    # print(df_rm.subnormal.value_counts())

    registro['quantidade_de_municipios_processados'] = df_rm.cod_municipio.unique().shape[0]

    setores_normais = df_rm.loc[filter & (df_rm.loc[filter].subnormal == 0)].shape[0]
    registro['quantidade_original_de_setores_normais'] = setores_normais
    
    setores_subnormais = df_rm.loc[filter & (df_rm.loc[filter].subnormal == 1)].shape[0]
    registro['quantidade_original_de_setores_subnormais'] = setores_subnormais
    # print(setores_normais, setores_subnormais)
    

    samples = min(setores_normais, math.floor(setores_subnormais * 1.9))
    # samples = 1

    # prepara amostras e lições
    x = df_rm.loc[(filter) & (subnormal)]
    x_ = df_rm.loc[(filter) & (~subnormal)].sample(n=samples, random_state=13)
    X = pd.concat([x, x_]).loc[(filter), dimensoes]
    transformer = Normalizer().fit(X)
    
    # X = transformer.transform(X)

    y = pd.concat([x, x_]).loc[(filter), 'subnormal']
    # X = df_rm.loc[(filter), dimensoes]
    # y = df_rm.loc[(filter), 'subnormal']

    # Processa os modelos, LDA
    # print(y.value_counts())
    clf = LinearDiscriminantAnalysis(solver='svd')
    clf.fit(X, y)
    # print(np.unique(clf.predict(X), return_counts=True))
    # predicao = clf.predict(transformer.transform(df_rm.loc[(filter), dimensoes]))
    predicao = clf.predict(df_rm.loc[(filter), dimensoes])
    try: 
        atribuidos_como_subnormais = np.unique(predicao, return_counts=True)[1][1]
    except:
        atribuidos_como_subnormais = 0
    registro['atribuidos_como_subnormais_LDA'] = atribuidos_como_subnormais
    # print(clf.score(X, y))
    registro['proporcao_atribuido_original_LDA'] = atribuidos_como_subnormais / setores_subnormais
    registro['score_LDA'] = clf.score(X, y)
    df.loc[(filter), 'subnormal_LDA'] = predicao

    # Processa model RandonForest
    clf = RandomForestClassifier(max_depth=10, random_state=0)
    clf.fit(X, y)
    # print(np.unique(clf.predict(X), return_counts=True))
    # predicao = clf.predict(transformer.transform(df_rm.loc[(filter), dimensoes]))
    predicao = clf.predict(df_rm.loc[(filter), dimensoes])
    try:
        atribuidos_como_subnormais = np.unique(predicao, return_counts=True)[1][1]
    except:
        atribuidos_como_subnormais = 0    
    registro['atribuidos_como_subnormais_RandomForest'] = atribuidos_como_subnormais
    # print(clf.score(X, y))
    registro['proporcao_atribuido_original_RandomForest'] = atribuidos_como_subnormais / setores_subnormais
    registro['score_RandomForest'] = clf.score(X, y)
    df.loc[(filter), 'subnormal_RandomForest'] = predicao


    # # Processa modelo SVC
    # clf = LinearSVC(random_state=0, tol=1e-5)
    # clf.fit(Normalizer().fit_transform(X), y)
    # # print(np.unique(clf.predict(X), return_counts=True))
    # predicao = clf.predict(Normalizer().fit_transform(df_rm.loc[(filter), dimensoes]))
    # try:
    #     atribuidos_como_subnormais = np.unique(clf.predict(Normalizer().fit_transform(df_rm.loc[(filter), dimensoes])), return_counts=True)[1][1]
    # except:
    #     atribuidos_como_subnormais = 0    
    # registro['atribuidos_como_subnormais_SVC'] = atribuidos_como_subnormais
    # # print(clf.score(X, y))
    # registro['proporcao_atribuido_original_SVC'] = atribuidos_como_subnormais / setores_subnormais
    # registro['score_SVC'] = clf.score(Normalizer().fit_transform(X), y)
    # df.loc[(filter), 'subnormal_SVC'] = predicao

    # Salva as predicoes em atributos
    return registro

In [104]:
# Processa cada um dos municípios da sua Região MEtropolitana
result = [] 

for rm in rms:

    rm_filter = (df.rm == rm)
    registro = processa_modelos(rm_filter, rm=rm)
    # Salva as predicoes em atributos
    result.append(registro)

(1666, 33)
(985, 33)
(2987, 33)
(2806, 33)
(842, 33)
(3143, 33)
(4907, 33)
(465, 33)
(14990, 33)
(21753, 33)
(2171, 33)
(3106, 33)
(3391, 33)
(5285, 33)
(1576, 33)
(3659, 33)


In [105]:
df.loc[df.rm.isin([13, 14, 15, 16]) & (df.subnormal == 0) & (df.subnormal_LDA == 1), ['rm', 'nome_rm', 'numero_de_domicilios', 'numero_moradores']].groupby('rm').agg({
    'nome_rm': 'first',
    'rm': 'count',
    'numero_de_domicilios': 'sum',
    'numero_moradores': 'sum'
})

,nome_rm,rm,numero_de_domicilios,numero_moradores
rm,,,,
13,Rio de Janeiro,1301,273085.0,978127.0
14,São Paulo,1746,374643.0,1476084.0
15,Baixada Santista,165,43319.0,167535.0
16,Campinas,229,41165.0,161620.0


In [ ]:
# result

In [ ]:
df.subnormal_LDA.value_counts()

In [ ]:
df.subnormal_RandomForest.value_counts()

In [ ]:
df.columns

In [ ]:
r_ufs = df[df.rm.isin(cidades_fora_rms)].loc[(subnormal), ['nome_UF', 'subnormal']].groupby('nome_UF').count().reset_index()

In [ ]:
r_ufs

In [ ]:
df_setores = df[df.rm.isin(cidades_fora_rms)].loc[:, ['nome_UF', 'subnormal', 'Cod_setor']].groupby('nome_UF').agg({'subnormal': 'sum', 'Cod_setor':'count'})

In [ ]:
df_setores[df_setores.subnormal < 20]

In [ ]:
df_setores[df_setores.subnormal < 20].sum()

In [ ]:
cidades_fora_rms

In [ ]:
r_ufs[r_ufs.subnormal >= 20]

In [ ]:
r_ufs[r_ufs.subnormal < 20]

In [ ]:
# df[(df.rm.isin(cidades_fora_rms))]

In [ ]:
# Processa cada uma das UFs
df_uf = df[df.rm.isin(cidades_fora_rms)]
ufs = r_ufs[r_ufs.subnormal >= 20].nome_UF.to_list()

for uf in ufs:

    uf_filter = (df.nome_UF == uf) & (df.rm.isin(cidades_fora_rms))
    print(uf)

    registro = processa_modelos(uf_filter, uf=uf)
    # Salva as predicoes em atributos
    result.append(registro)


In [ ]:
gr_filter = (df.cod_grande_regiao == 4.) & (df.subnormal_LDA.isna())

In [ ]:
## TODO
## Processar as cidades restantes por MacroRegião 
## Região 4 (Sul) não possui licoes suficientes de municípios com setores subnormais
for gr in [1., 2., 3., 5.]:
    gr_filter = (df.cod_grande_regiao == gr) & (~df.processado)
    print(gr)

    if len(df[gr_filter]) > 0:
        registro = processa_modelos(gr_filter, uf=f'Macro Região {str(gr)}')
        # Salva as predicoes em atributos
        result.append(registro)

In [ ]:
df[df.cod_grande_regiao == 4.]

In [ ]:
pd.DataFrame(result).to_excel('resultados/geral_por_RM_UF.xlsx')

In [ ]:
pd.DataFrame(result).quantidade_de_municipios_processados.sum()

In [ ]:
df.subnormal_LDA.value_counts()

In [ ]:
df.subnormal_RandomForest.value_counts()
# df.columns

In [ ]:
df.subnormal.value_counts()

In [ ]:
df.to_csv('resultados/modelo_preditivo.csv.zip', compression='zip')

In [ ]:
df[df.subnormal_LDA.isna()]

In [ ]:
df

In [ ]:
df.processado.value_counts()